# calculates the seasonal anomalies and the seasonal tercile categories from the gridded VCSN monthly files, using the NZ 6 regions shapefiles 

In [1]:
# Paramaters 

# var_name = 'Rain_bc'
# var_name = 'Tmin_N'
# var_name = 'Tmax_N'
# var_name = 'Tmean_N'
# var_name = 'SoilM'
# var_name = 'Wind'
var_name = 'Rad'

in ['Agent', 'Lat', 'Longt', 'Date', 'MSLP', 'PET', 'Rain', 'RH', 'SoilM',
       'ETmp', 'Rad', 'TMax', 'Tmin', 'VP', 'Wind', 'Rain_bc', 'Tmax_N',
       'Tmin_N']

In [2]:
import os
import sys
import pathlib

In [3]:
%matplotlib inline
from matplotlib import pyplot as plt

import numpy as np
import pandas as pd
from scipy.spatial import cKDTree
from itertools import product

In [4]:
import salem
import geopandas as gpd

In [5]:
import xarray as xr

### function to calculate the anomalies with respect to the 1981 - 2010 climatology 

In [6]:
def demean(x): 
    return x - x.loc['1981':'2010',].mean()

In [7]:
var_name

'Rad'

In [8]:
big_var = var_name.split('_')[0].upper()

In [9]:
HOME = pathlib.Path.home()

In [10]:
dpath = HOME / 'operational/VCSN/data/NC/MONTHLY/' / var_name.upper()

In [11]:
var_name.upper()

'RAD'

In [12]:
dpath

PosixPath('/home/nicolasf/operational/VCSN/data/NC/MONTHLY/RAD')

In [13]:
dset = salem.open_xr_dataset(dpath / f'VCSN_gridded_{var_name}_1979-01_2019-12.nc') 

In [14]:
dset

<xarray.Dataset>
Dimensions:  (lat: 257, lon: 241, time: 492)
Coordinates:
  * time     (time) datetime64[ns] 1979-01-31 1979-02-28 ... 2019-12-31
  * lat      (lat) float64 -47.27 -47.23 -47.17 -47.12 ... -34.52 -34.48 -34.42
  * lon      (lon) float64 166.5 166.5 166.6 166.6 ... 178.3 178.4 178.4 178.5
Data variables:
    Rad      (time, lat, lon) float64 ...
    agent    (lat, lon) float32 ...
Attributes:
    pyproj_srs:  +units=m +proj=latlong +datum=WGS84

### calculates the seasonal average (or sum if Rain_bc is the variable )

In [15]:
if var_name == 'Rain_bc': 
    dset = dset.rolling(time=3, min_periods=3).sum()
else: 
    dset = dset.rolling(time=3, min_periods=3).mean()

In [16]:
dset = dset.isel(time=slice(2,None))

In [17]:
nz_regions = gpd.read_file(HOME / 'research' / 'Smart_Ideas' / 'data' / 'shapefiles' / 'NZ_regions' / 'NZ_6_regions' / 'NZ_regions_corrected.shp') 

In [18]:
nz_regions

,OBJECTID,Id,gridcode,Shape_Leng,Shape_Area,Location,geometry
0,1,1,1,85.215338,5.032753,NNI,"MULTIPOLYGON (((174.70530 -38.17377, 174.70545..."
1,2,2,2,12.336015,2.994028,WNI,"MULTIPOLYGON (((175.13516 -41.37745, 175.13507..."
2,3,3,3,14.235493,3.775388,ENI,"MULTIPOLYGON (((175.85595 -41.35970, 175.85595..."
3,4,4,4,34.656463,3.066280,NSI,"MULTIPOLYGON (((171.32620 -42.12355, 171.32602..."
4,5,5,6,20.191504,4.827228,ESI,"MULTIPOLYGON (((170.21675 -46.05955, 170.21609..."
5,6,6,5,42.941379,9.057410,WSI,"MULTIPOLYGON (((169.20749 -46.66371, 169.20742..."


In [19]:
nz_regions = nz_regions.to_crs(epsg=4326)

In [20]:
opath_root = HOME / 'research' / 'Smart_Ideas' / 'outputs' / 'targets' / 'NZ_regions' / 'NZ_6_regions'

In [21]:
if not opath_root.exists(): 
    opath_root.mkdir(parents=True)

### defines the number of quantiles we want 

In [22]:
num_quantiles = 3

In [23]:
quant_values = np.linspace(0, 1, num_quantiles + 1, endpoint=True)

In [24]:
quant_values = quant_values[1:-1]

In [25]:
quant_values

array([0.33333333, 0.66666667])

In [26]:
col_labs = [f"Q{int(x)}" for x in (quant_values*100)]

In [27]:
col_labs

['Q33', 'Q66']

In [28]:
# f, axes = plt.subplots(nrows=3, ncols=2)
# axes = axes.flatten()

quantiles_dict = {}

for i, region_name in enumerate(['NNI','ENI','WNI','NSI','WSI','ESI']): 
    
    shape = nz_regions.query(f"Location == '{region_name}'")
    
    region = dset.salem.subset(shape=shape)

    region = region.salem.roi(shape=shape, all_touched=True)
    
#     region[var_name].isel(time=0).plot(ax=axes[i])
    
    ts = region.mean(dim=['lat','lon'])
    
    ts_df = ts[var_name].to_dataframe()
            
    ts_series = ts_df.loc[:,var_name]
    
    ts_series_cat = []
    
    quantiles_list = []
    
    for month in range(1, 13):
        
        ts_series_m = ts_series[ts_series.index.month == month]
        
        clim = ts_series_m.loc['1981':'2010']
        
        quantiles = [clim.quantile(q=q) for q in quant_values.tolist()]
        
        quantiles_list.append(quantiles.copy())
        
        quantiles.insert(0, -np.inf)
        
        quantiles.append(np.inf)
        
        ts_series_m_cats = pd.cut(ts_series_m, quantiles, labels=list(range(1, num_quantiles + 1)))
        
        ts_series_cat.append(ts_series_m_cats)
        
        del(quantiles)
     
    quantiles_dict[region_name]  = np.array(quantiles_list)
    
    ts_series_cat = pd.concat(ts_series_cat, axis=0)
    
    ts_series_cat = ts_series_cat.sort_index()
    
    ts_df.loc[:,f'cat_{num_quantiles}'] = ts_series_cat
    
    ts_df.loc[:,'anomalies'] = ts_df.loc[:,var_name].groupby(ts_df.index.month).apply(demean)
    
    opath = opath_root / big_var / region_name 
    
    if not opath.exists(): 
        opath.mkdir(parents=True)
        
    ts_df.to_csv(opath / f'TS_NZ_region_{region_name}_{big_var}_{num_quantiles}_quantiles_anoms.csv')
    
    #descriptive statistics per quantile category 
    
    ts_df.groupby(ts_df.loc[:,f'cat_{num_quantiles}']).describe().to_csv(opath / f'descriptive_stats_{region_name}__{big_var}_{num_quantiles}.csv')
    
    print(f"region {region_name} processed for variable {big_var}")
    

region NNI processed for variable RAD
region ENI processed for variable RAD
region WNI processed for variable RAD
region NSI processed for variable RAD
region WSI processed for variable RAD
region ESI processed for variable RAD


### saves the climatological terciles calculated from the VCSN regional aggregates

In [29]:
quantiles_list = []
for region_name in ['NNI','ENI','WNI','NSI','WSI','ESI']: 
    df = pd.DataFrame(quantiles_dict[region_name])
    df.index = range(1, 13)
    df.index.name = 'season'
    df.columns = pd.MultiIndex.from_product([[region_name],col_labs])
    quantiles_list.append(df)

In [30]:
quantiles_df = pd.concat(quantiles_list, axis=1)

In [31]:
quantiles_df

NNI                   ENI                   WNI             \
              Q33        Q66        Q33        Q66        Q33        Q66   
season                                                                     
1       20.819861  21.448777  21.272664  22.093260  21.061377  21.954414   
2       20.480753  21.125412  20.665280  21.526482  21.078526  21.689622   
3       18.572889  18.958652  18.603052  19.332622  19.013971  19.439964   
4       15.025882  15.663719  14.744609  15.530685  15.031409  15.667694   
5       11.358929  11.905582  10.866251  11.288323  11.044009  11.461646   
6        8.391971   8.798104   7.900180   8.078208   7.782832   8.125428   
7        7.102749   7.360209   6.291280   6.563954   6.243780   6.523703   
8        7.591970   7.857803   6.862727   7.224760   6.737661   7.022261   
9        9.640261  10.085765   9.260263   9.588825   8.921246   9.418377   
10      12.782385  13.270705  12.838525  13.245889  12.157535  12.691356   
11      16.124317  16.841248  16.448748  17.237479  15.766186  16.616366   
12      18.794326  19.676577  19.506112  20.327347  18.926835  19.691404   

              NSI                   WSI                   ESI             
              Q33        Q66        Q33        Q66        Q33        Q66  
season                                                                    
1       21.189076  22.129736  20.154535  20.865758  20.922953  21.672560  
2       21.043352  21.626373  19.806858  20.562944  20.032605  20.962803  
3       18.832629  19.492998  17.472209  17.852039  17.412796  18.277333  
4       15.032954  15.511761  13.562990  13.992223  13.715287  14.449117  
5       10.939759  11.288740   9.324501   9.602324   9.892127  10.260901  
6        7.473459   7.837181   6.090252   6.357443   6.795269   6.950532  
7        6.041460   6.227390   4.729674   4.932998   5.303289   5.529248  
8        6.590289   6.753570   5.486626   5.622386   5.912735   6.098620  
9        8.963753   9.318676   7.721255   8.108533   8.357364   8.691005  
10      12.496631  13.039063  11.372999  11.867254  12.098492  12.697809  
11      16.345861  17.191047  15.288408  16.090857  16.006178  16.824448  
12      19.238440  20.169825  18.482232  19.176596  19.078350  19.997285

In [32]:
quantiles_df.to_csv(opath.parent / f'Climatological_quantiles_{num_quantiles}_cat_{big_var}.csv')